# Jira Satisfaction Analysis (BERT)
## Instructions
1. **Secrets**: Add JIRA_URL, JIRA_EMAIL, JIRA_API_TOKEN in the sidebar (Key icon).
2. **Runtime**: Change Runtime type to GPU (Runtime > Change runtime type > T4 GPU).
3. **Run**: Click the Play button below.


In [ ]:
# @title Jira Satisfaction Analysis (All-in-One)
# COPY THIS ENTIRE SCRIPT INTO A GOOGLE COLAB CELL
# ==============================================================================
# 1. SETUP & INSTALLATION
# ==============================================================================
import os
import sys

def install_dependencies():
    print("🚀 Installing dependencies...")
    os.system('pip install transformers torch pandas requests scikit-learn python-dotenv joblib')
    print("✅ Dependencies installed.")

# Check if running in Colab to install dependencies automatically
if 'google.colab' in sys.modules:
    install_dependencies()
    from google.colab import drive
    from google.colab import userdata
    
    # Mount Drive
    print("📂 Mounting Google Drive...")
    drive.mount('/content/drive')
    
    # Set Paths for Colab
    BASE_DIR = '/content/drive/MyDrive/jira_analysis'
    os.makedirs(BASE_DIR, exist_ok=True)
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    MODELS_DIR = os.path.join(BASE_DIR, 'models/bert')
    
    # Get Secrets (Make sure to add these in Colab Secrets tab)
       try:
        JIRA_URL = userdata.get('JIRA_URL')
        JIRA_EMAIL = userdata.get('JIRA_EMAIL')
        JIRA_API_TOKEN = userdata.get('JIRA_API_TOKEN')
    except:
        print("⚠️ Secrets not found! Please add JIRA_URL, JIRA_EMAIL, JIRA_API_TOKEN in Colab Secrets.")
        JIRA_URL = "https://certifyos.atlassian.net/"
        JIRA_EMAIL = "anurag.rai@certifyos.com"
        JIRA_API_TOKEN = "ATATT3xFfGF0BK0DPRifn8dXP3AnH0fS8TKfH-QCU5Ah_h3JxSqxKo0QryD1A0B37gydyLp4QMMPLejMxVbQj9IcsObdIPyF3v8t3y4mTX07XQuCH7B0uBSQFUlkGUMj3IzbUsFx21yBGL-6w0bzXfm2D_XJvsQFBe8a-AF-5epvchUeCMpmAL8=47A52710"
else:

    # Local fallback
    BASE_DIR = '.'
    DATA_DIR = 'data'
    MODELS_DIR = 'models/bert'
    JIRA_URL = os.getenv("JIRA_URL")
    JIRA_EMAIL = os.getenv("JIRA_EMAIL")
    JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")

# Debug Config
print(f"🔧 Config Check:")
print(f"   URL: {JIRA_URL}")
print(f"   Email: {JIRA_EMAIL}")
print(f"   Token: {'*' * 5}{JIRA_API_TOKEN[-4:] if JIRA_API_TOKEN else 'None'}")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# ==============================================================================
# IMPORT UTILS
# ==============================================================================
def load_data(file_path):
    """Load data from CSV or Excel file."""
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        print(f"📄 Detected Excel file. Converting {file_path}...")
        try:
            df = pd.read_excel(file_path)
            # Save as CSV for consistency in future steps
            csv_path = file_path.replace('.xlsx', '.csv').replace('.xls', '.csv')
            df.to_csv(csv_path, index=False)
            print(f"✅ Converted to CSV: {csv_path}")
            return df, csv_path
        except Exception as e:
            print(f"❌ Error reading Excel file: {e}")
            return None, None
    elif file_path.endswith('.gsheet'):
        print(f"📄 Detected Google Sheet: {file_path}")
        try:
            # Colab-specific auth
            from google.colab import auth
            auth.authenticate_user()
            import gspread
            from google.auth import default
            
            creds, _ = default()
            gc = gspread.authorize(creds)
            
            # Extract filename without path and extension
            filename = os.path.basename(file_path).replace('.gsheet', '')
            print(f"   Connecting to Google Sheet: '{filename}'...")
            
            # Open by title
            worksheet = gc.open(filename).sheet1
            rows = worksheet.get_all_values()
            
            # Convert to DataFrame
            df = pd.DataFrame(rows[1:], columns=rows[0])
            
            # Save as CSV for consistency
            csv_path = file_path.replace('.gsheet', '.csv')
            df.to_csv(csv_path, index=False)
            print(f"✅ Converted to CSV: {csv_path}")
            return df, csv_path
        except Exception as e:
            print(f"❌ Error reading Google Sheet. Ensure you are authenticated and the sheet name matches exactly.")
            print(f"Error: {e}")
            return None, None
    elif file_path.endswith('.csv'):
        try:
            df = pd.read_csv(file_path)
            return df, file_path
        except Exception as e:
            print(f"❌ Error reading CSV file: {e}")
            return None, None
    else:
        print(f"❌ Unsupported file format: {file_path}")
        return None, None

# ==============================================================================
# 2. IMPORTS & CONFIG
# ==============================================================================
import pandas as pd
import numpy as np
import torch
import re
import time
import requests
import joblib
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ThreadPoolExecutor, as_completed

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ Using Device: {DEVICE}")

# ==============================================================================
# 3. JIRA FETCHER (Extract Data)
# ==============================================================================
class JiraFetcher:
    def __init__(self):
        self.session = requests.Session()
        self.session.auth = (JIRA_EMAIL, JIRA_API_TOKEN)
        self.session.headers.update({'Content-Type': 'application/json', 'Accept': 'application/json'})

    def get_ticket_details(self, issue_key):
        url = f"{JIRA_URL}/rest/api/3/issue/{issue_key}"
        params = {'fields': 'summary,comment', 'expand': 'renderedFields'}
        
        retries = 0
        max_retries = 5
        base_wait = 5
        
        while retries <= max_retries:
            try:
                response = self.session.get(url, params=params, timeout=30)
                if response.status_code == 200:
                    return response.json()
                elif response.status_code == 429:
                    wait_time = base_wait * (2 ** retries)
                    print(f"⚠️ Rate limited on {issue_key}. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"❌ Failed to fetch {issue_key}: Status {response.status_code}")
                    return None
            except Exception as e:
                print(f"Error fetching {issue_key}: {e}")
                return None
        
        print(f"❌ Max retries reached for {issue_key}")
        return None

    def extract_conversation(self, issue_data):
        if not issue_data: return ""
        fields = issue_data.get('fields', {})
        text = fields.get('summary', '') + " "
        comments = fields.get('comment', {}).get('comments', [])
        for comment in comments:
            body = comment.get('body', {})
            text += self._extract_text(body) + " "
        return re.sub(r'\s+', ' ', text).strip()

    def _extract_text(self, body):
        text_parts = []
        if isinstance(body, dict) and 'content' in body:
            for node in body['content']:
                if 'content' in node:
                    for text_node in node['content']:
                        if 'text' in text_node:
                            text_parts.append(text_node['text'])
        return " ".join(text_parts)

# ==============================================================================
# 4. HEURISTIC LABELING (Cold Start)
# ==============================================================================
def get_heuristic_label(text):
    text = str(text).lower()
    positive = ["thank", "resolved", "fixed", "appreciate", "great", "good", "working now"]
    negative = ["not working", "fail", "error", "broken", "bad", "terrible", "frustrated", "delay"]
    
    if any(kw in text for kw in ["working now", "resolved", "fixed"]) and not any(kw in text for kw in ["not resolved", "not fixed"]):
        return "Positive"
    if any(kw in text for kw in positive): return "Positive"
    if any(kw in text for kw in negative): return "Negative"
    return "Neutral"

def create_training_data(file_path):
    print(f"🏷️ Creating training data from {file_path}...")
    df, csv_path = load_data(file_path)
    if df is None: return None
    
    # 1. Fetch Text if not present
    if 'text' not in df.columns:
        print("   Fetching ticket text from Jira...")
        jira = JiraFetcher()
        texts = []
        try:
            keys_col = 'Key' if 'Key' in df.columns else df.columns[0]
            # Limit for demo/speed check
            keys = df[keys_col].tolist()
            
            # Sequential processing to strictly avoid rate limits
            print(f"   (Switching to sequential fetching to avoid 429 errors. Total: {len(keys)})")
            
            success_count = 0
            for i, key in enumerate(keys):
                try:
                    # Log every 10 tickets or important status
                    if i % 5 == 0: 
                        print(f"   ⏳ Processing {i+1}/{len(keys)}: {key}...", end='\r')
                        
                    data = jira.get_ticket_details(key)
                    if data:
                        # Extract Summary + Comments
                        txt = jira.extract_conversation(data)
                        texts.append({'key': key, 'text': txt})
                        success_count += 1
                    
                    # Consistent delay between every request
                    time.sleep(1.0) 
                except Exception as e:
                    print(f"   ❌ Error on {key}: {e}")
            
            print(f"\n   ✅ Finished fetching. Success: {success_count}/{len(keys)}")
            
            text_df = pd.DataFrame(texts)
            df = df.merge(text_df, left_on=keys_col, right_on='key', how='left')
        except Exception as e:
            print(f"❌ Error fetching data: {e}")
            return None

    # 2. Apply Heuristics
    print("🧠 Applying Heuristic Rules (Keyword Logic)...")
    df['label'] = df['text'].apply(get_heuristic_label)
    output_path = os.path.join(DATA_DIR, 'labeled_training_data.csv')
    df.to_csv(output_path, index=False)
    print(f"\n✅ Saved labeled data to {output_path}")
    print(df['label'].value_counts())
    return output_path

# ==============================================================================
# 5. BERT TRAINING
# ==============================================================================
class SatisfactionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.texts)
    def __getitem__(self, item):
        encoding = self.tokenizer.encode_plus(
            str(self.texts[item]), max_length=self.max_len, padding='max_length',
            truncation=True, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[item], dtype=torch.long)
        }

def train_bert(data_path):
    print("\n🏋️ Starting BERT Training...")
    # Keep default na values but convert empty text to NaN so we can drop it
    df = pd.read_csv(data_path)
    df = df.dropna(subset=['text', 'label'])
    
    if len(df) == 0:
        print("❌ Error: Dataset is empty after filtering! Text fetching likely failed.")
        print("   Check your JIRA_URL/EMAIL/TOKEN secrets and try again.")
        return

    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    joblib.dump(le, os.path.join(MODELS_DIR, 'label_encoder.joblib'))
    
    train_df, val_df = train_test_split(df, test_size=0.2)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    train_ds = SatisfactionDataset(train_df.text.values, train_df.label_encoded.values, tokenizer, 128)
    val_ds = SatisfactionDataset(val_df.text.values, val_df.label_encoded.values, tokenizer, 128)
    
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
    model = model.to(DEVICE)
    
    optimizer = AdamW(model.parameters(), lr=2e-5)
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    
    print(f"   Training on {len(train_df)} samples, Validating on {len(val_df)} samples.")
    
    model.train()
    for epoch in range(3):
        total_loss = 0
        print(f"   Epoch {epoch+1}/3...", end=' ')
        batch_count = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(DEVICE)
            mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            outputs = model(input_ids, attention_mask=mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            batch_count += 1
            
        avg_loss = total_loss / len(train_loader)
        print(f"Loss: {avg_loss:.4f}")
        
    model.save_pretrained(MODELS_DIR)
    tokenizer.save_pretrained(MODELS_DIR)
    print("✅ Model trained and saved!")

# ==============================================================================
# 6. PREDICTION ON FULL SHEET
# ==============================================================================
def predict_full_sheet(file_path):
    print(f"\n🔮 Generating Predictions for {file_path}...")
    tokenizer = BertTokenizer.from_pretrained(MODELS_DIR)
    model = BertForSequenceClassification.from_pretrained(MODELS_DIR)
    model = model.to(DEVICE)
    model.eval()
    le = joblib.load(os.path.join(MODELS_DIR, 'label_encoder.joblib'))
    
    df, csv_path = load_data(file_path)
    if df is None: return

    jira = JiraFetcher()
    
    # Needs text column first
    if 'text' not in df.columns:
        print("   Fetching text for prediction (this might take time)...")
        # Logic to fetch text would go here similar to create_training_data
        # For brevity assuming create_training_data was run or we fetch on fly
        # (Reusing fetch logic needed if new file)
    
    # Prediction Loop
    predictions = []
    confidences = []
    
    # Process batching for speed
    texts = df['text'].tolist() if 'text' in df.columns else [] # Placeholder
    
    # If text is missing, we must fetch it. 
    # For independent running, we assume the user provides a CSV that needs processing.
    # We will implement a simple sequential fetch-predict for robustness in Colab
    
    keys_col = 'Key' if 'Key' in df.columns else df.columns[0]
    keys = df[keys_col].tolist()
    
    for i, key in enumerate(keys):
        try:
            # Check if text exists, else fetch
            text = df.loc[i, 'text'] if 'text' in df.columns else None
            if not isinstance(text, str):
                details = jira.get_ticket_details(key)
                text = jira.extract_conversation(details)
            
            inputs = tokenizer(str(text), return_tensors='pt', max_length=128, truncation=True, padding='max_length')
            with torch.no_grad():
                outputs = model(inputs['input_ids'].to(DEVICE), attention_mask=inputs['attention_mask'].to(DEVICE))
                probs = torch.nn.functional.softmax(outputs.logits, dim=1)
                conf, pred_idx = torch.max(probs, dim=1)
            
            pred_label = le.inverse_transform([pred_idx.item()])[0]
            predictions.append(pred_label)
            confidences.append(conf.item())
            
            if i % 10 == 0: print(f"   Processed {i}/{len(keys)}", end='\r')
            time.sleep(0.5) # Proactive rate limiting
            
        except Exception as e:
            predictions.append("Error")
            confidences.append(0.0)
            
    df['Predicted_Satisfaction'] = predictions
    df['Confidence'] = confidences
    
    out_path = csv_path.replace('.csv', '_bert_colab_predictions.csv')
    df.to_csv(out_path, index=False)
    print(f"\n✅ Predictions saved to {out_path}")

# ==============================================================================
# 7. EXECUTION BLOCK
# ==============================================================================
if __name__ == "__main__":
    # Example Workflow
    print("Select an action:")
    print("1. Label Data & Train Model")
    print("2. Predict on New CSV")
    
    # In Colab, you can just change this variable manually
    ACTION = "2" 
    
    if 'google.colab' in sys.modules:
        print(f"📂 Files in {DATA_DIR}:")
        try:
            files = os.listdir(DATA_DIR)
            for f in files: print(f"   - {f}")
        except:
            print(f"   (Could not list files in {DATA_DIR})")
    
    INPUT_CSV = os.path.join(DATA_DIR, "Sheet for 3_months - Sheet1.csv") # Default
    
    # Auto-detect if default missing
    if not os.path.exists(INPUT_CSV) and os.path.exists(DATA_DIR):
        print(f"⚠️ Default file {INPUT_CSV} not found.")
        try:
            # Find any file containing 'Sheet for 3_months' or just pick the first .gsheet/.csv
            candidates = [f for f in os.listdir(DATA_DIR) if (f.endswith('.csv') or f.endswith('.gsheet')) and 'predictions' not in f and 'labeled' not in f]
            if candidates:
                # Prefer one with "Sheet for 3_months"
                better_candidates = [f for f in candidates if "Sheet for 3_months" in f]
                if better_candidates:
                    chosen = better_candidates[0]
                else:
                    chosen = candidates[0]
                    
                INPUT_CSV = os.path.join(DATA_DIR, chosen)
                print(f"✅ Auto-selected valid file: {INPUT_CSV}")
        except:
            pass

    # Allow user to override input filename easily
    print(f"\n📝 Using Input File: {INPUT_CSV}")
    print(f"   (If this is wrong, edit the INPUT_CSV variable in the code)")
    
    if ACTION == "1":
        labeled_path = create_training_data(INPUT_CSV)
        if labeled_path:
            train_bert(labeled_path)
            predict_full_sheet(INPUT_CSV) # Predict on the same sheet after training
            
    elif ACTION == "2":
        predict_full_sheet(INPUT_CSV)
